In [22]:
import torch
from ultralytics import YOLO
import numpy as np

In [23]:
# NMS 함수 정의
def apply_nms(predictions, iou_threshold=0.5, score_threshold=0.3):
    """
    Apply Non-Maximum Suppression (NMS) to YOLO predictions.
    Args:
        predictions: List of [x1, y1, x2, y2, score, class_id] per detection
        iou_threshold: IoU threshold for NMS
        score_threshold: Score threshold to filter weak detections
    Returns:
        nms_results: Filtered detections after applying NMS
    """
    if len(predictions) == 0:
        return []

    predictions = np.array(predictions)
    x1, y1, x2, y2, scores, class_ids = (
        predictions[:, 0],
        predictions[:, 1],
        predictions[:, 2],
        predictions[:, 3],
        predictions[:, 4],
        predictions[:, 5],
    )

    # Sort detections by score
    indices = scores.argsort()[::-1]
    selected_boxes = []

    while len(indices) > 0:
        current = indices[0]
        selected_boxes.append(predictions[current])

        # Calculate IoU with the remaining boxes
        xx1 = np.maximum(x1[current], x1[indices[1:]])
        yy1 = np.maximum(y1[current], y1[indices[1:]])
        xx2 = np.minimum(x2[current], x2[indices[1:]])
        yy2 = np.minimum(y2[current], y2[indices[1:]])

        inter_area = np.maximum(0, xx2 - xx1) * np.maximum(0, yy2 - yy1)
        box_area_current = (x2[current] - x1[current]) * (y2[current] - y1[current])
        box_area_indices = (x2[indices[1:]] - x1[indices[1:]]) * (y2[indices[1:]] - y1[indices[1:]])
        union_area = box_area_current + box_area_indices - inter_area

        iou = inter_area / (union_area + 1e-6)
        # Retain boxes with IoU less than the threshold
        indices = indices[1:][iou < iou_threshold]

    # Convert results to list format and filter by score threshold
    nms_results = [box.tolist() for box in selected_boxes if box[4] > score_threshold]
    return nms_results

In [24]:
# YOLOv8 모델 로드
model = YOLO('yolov8n.pt')  # Pretrained YOLOv8 모델 사용

In [25]:
# 데이터셋 경로
dataset_yaml = '/home/ec2-user/SageMaker/data/yolov8detections/data.yaml'

In [ ]:
# YOLO 모델 훈련 (필요 시 주석 제거)
model.train(data=dataset_yaml, epochs=10, batch=16)

Ultralytics 8.3.39 🚀 Python-3.10.14 torch-2.2.2 CUDA:0 (Tesla T4, 14918MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/ec2-user/SageMaker/data/yolov8detections/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, s

train: Scanning /home/ec2-user/SageMaker/data/yolov8detections/train/labels.cache... 816 images, 1 backgrounds, 0 corrupt: 100%|██████████| 816/816 [00:00<?, ?it/s]
val: Scanning /home/ec2-user/SageMaker/data/yolov8detections/valid/labels.cache... 232 images, 0 backgrounds, 0 corrupt: 100%|██████████| 232/232 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 


In [20]:
# 테스트 데이터 경로
test_path = '/home/ec2-user/SageMaker/data/yolov8detections/test/labels'

In [21]:
# 예측 수행
results = model.predict(source=test_path, save=False, conf=0.1)

FileNotFoundError: No images or videos found in /home/ec2-user/SageMaker/data/yolov8detections/test/labels. Supported formats are:
images: {'pfm', 'png', 'webp', 'dng', 'jpeg', 'bmp', 'tif', 'jpg', 'mpo', 'heic', 'tiff'}
videos: {'mpg', 'webm', 'mov', 'gif', 'mpeg', 'mp4', 'ts', 'mkv', 'avi', 'wmv', 'm4v', 'asf'}

In [19]:
# NMS 적용
final_results = []
for result in results:
    # Convert YOLO output to format [x1, y1, x2, y2, score, class_id]
    predictions = []
    for box in result.boxes.xyxy:  # Access bounding boxes in xyxy format
        x1, y1, x2, y2 = box[:4].tolist()
        score = box[4].item()
        class_id = box[5].item()
        predictions.append([x1, y1, x2, y2, score, class_id])

    # Apply NMS
    nms_result = apply_nms(predictions, iou_threshold=0.5, score_threshold=0.3)
    final_results.append(nms_result)

IndexError: index 4 is out of bounds for dimension 0 with size 4

In [21]:
# 결과 출력
for i, img_results in enumerate(final_results):
    print(f"Image {i+1} detections after NMS:")
    for det in img_results:
        print(det)

Image 1 detections after NMS:
Image 2 detections after NMS:
Image 3 detections after NMS:
Image 4 detections after NMS:
Image 5 detections after NMS:
Image 6 detections after NMS:
Image 7 detections after NMS:
Image 8 detections after NMS:
Image 9 detections after NMS:
Image 10 detections after NMS:
Image 11 detections after NMS:
Image 12 detections after NMS:
Image 13 detections after NMS:
Image 14 detections after NMS:
Image 15 detections after NMS:
Image 16 detections after NMS:
Image 17 detections after NMS:
Image 18 detections after NMS:
Image 19 detections after NMS:
Image 20 detections after NMS:
Image 21 detections after NMS:
Image 22 detections after NMS:
Image 23 detections after NMS:
Image 24 detections after NMS:
Image 25 detections after NMS:
Image 26 detections after NMS:
Image 27 detections after NMS:
Image 28 detections after NMS:
Image 29 detections after NMS:
Image 30 detections after NMS:
Image 31 detections after NMS:
Image 32 detections after NMS:
Image 33 detectio